Заглушка

In [1]:
!pip install datasets transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 111.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 21.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
import numpy as np
import pandas as pd

In [3]:
pd.options.display.max_colwidth = 300

In [4]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [5]:
model_name = "bankholdup/rugpt3_song_writer"

In [6]:
df_rec = pd.read_json('../content/sample_data/dataset.jsonl', lines=True).set_index('id')

In [7]:
df_rec

,date,rating,text
id,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/"
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!"
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам"""
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.
...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\nyyy: намекни хотя бы, это география или порнуха?\nxxx: это бл#%^ job is done!"
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. Начал штудировать теорию. Выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. Решил с утра начать бегать. Утром проснулся, попытался встать..., понял, что у меня телослож..."
463646,2020-11-26 06:46:00+00:00,88.0,"#всебожьяроса\nxxx: Судьба айтишников вообще незавидная\nxxx: Вы должны за копейки обслуживать все компы в организации\nxxx: Вы должны за копейки поддерживать всех сотрудников организации\nxxx: Но самое главное\nxxx: Руководство получает большие бабки и строит из себя ""эффективных менеджеров""\nx..."


In [8]:
df_rec.shape

(81497, 3)

In [9]:
import re

def clear_text(text):
    clr_text = re.sub(r"<.*?>", " ", text).lower()
    clr_text = summary = re.sub(r"\s", " ", clr_text)
    return clr_text

In [10]:
df_rec["clear_text"] = df_rec["text"].apply(lambda x: clear_text(x))
df_rec

,date,rating,text,clear_text
id,,,,
1,2004-08-30 11:24:00+00:00,22010.0,"<Ares> ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)","ppdv, все юниксы очень дружелюбны.. они просто очень разборчивы в друзьях ;)"
2,2004-08-30 11:25:00+00:00,25105.0,"<томатик_рад> а ты не чувствуешь красоту мира?\n<fox> честно говоря, я сейчас чувствую только отсутствие http.\n<томатик_рад> не туда смотришь, глянь вокруг!\n<fox> как я гляну, если http не работает? :/","а ты не чувствуешь красоту мира? честно говоря, я сейчас чувствую только отсутствие http. не туда смотришь, глянь вокруг! как я гляну, если http не работает? :/"
3,2004-08-30 11:27:00+00:00,7192.0,"<Дор> ""мышка, почему у тебя такие большие глаза?"" УЙДИ!!! я ХАРАКИРИ делаю!!!!!!","""мышка, почему у тебя такие большие глаза?"" уйди!!! я харакири делаю!!!!!!"
4,2004-08-30 11:28:00+00:00,29169.0,"<PPDV[os2]> ""Мальчики, вы что больные, бегать в палату к девочкам?! - Если б мы были больные - мы б бегали к другим мальчикам""","""мальчики, вы что больные, бегать в палату к девочкам?! - если б мы были больные - мы б бегали к другим мальчикам"""
5,2004-08-30 11:26:00+00:00,7140.0,<Ohtori_Akio> мы - как разработчики - живём с субейзом под одбц. \n<Ohtori_Akio> лучше бы мы жили в пещере с гоблинами.,мы - как разработчики - живём с субейзом под одбц. лучше бы мы жили в пещере с гоблинами.
...,...,...,...,...
463644,2020-11-26 06:12:00+00:00,698.0,"xxx: угадайте не гугля, что такое жопиздан!\nyyy: намекни хотя бы, это география или порнуха?\nxxx: это бл#%^ job is done!","xxx: угадайте не гугля, что такое жопиздан! yyy: намекни хотя бы, это география или порнуха? xxx: это бл#%^ job is done!"
463645,2020-11-26 06:12:00+00:00,816.0,"xxx:\nПосетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. Начал штудировать теорию. Выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. Решил с утра начать бегать. Утром проснулся, попытался встать..., понял, что у меня телослож...","xxx: посетила шальная мысль заняться собой, жирок сбросить и подкачаться малость. начал штудировать теорию. выяснил, что у человека бывает три типа телосложения - эктоморфное, мезоморфное и эндоморфное. решил с утра начать бегать. утром проснулся, попытался встать..., понял, что у меня телосложе..."
463646,2020-11-26 06:46:00+00:00,88.0,"#всебожьяроса\nxxx: Судьба айтишников вообще незавидная\nxxx: Вы должны за копейки обслуживать все компы в организации\nxxx: Вы должны за копейки поддерживать всех сотрудников организации\nxxx: Но самое главное\nxxx: Руководство получает большие бабки и строит из себя ""эффективных менеджеров""\nx...","#всебожьяроса xxx: судьба айтишников вообще незавидная xxx: вы должны за копейки обслуживать все компы в организации xxx: вы должны за копейки поддерживать всех сотрудников организации xxx: но самое главное xxx: руководство получает большие бабки и строит из себя ""эффективных менеджеров"" xxx: т...."


In [11]:
data = df_rec.loc[:, 'clear_text']

In [12]:
import re
from sklearn.model_selection import train_test_split

def build_text_files(data_json, dest_path):
    f = open(dest_path, 'w')
    data = ''
    for texts in data_json:
        summary = str(texts).strip()
#         summary = re.sub(r"<.*?>", " ", summary)
#         summary = re.sub(r"\s", " ", summary)
        data += summary + "  "
    f.write(data)

In [13]:
train, test = train_test_split(data, test_size=0.15)

build_text_files(train,'train_dataset.txt')
build_text_files(test,'test_dataset.txt')

In [14]:
print("Train dataset length: "+ str(len(train)))
print("Test dataset length: "+ str(len(test)))

Train dataset length: 69272
Test dataset length: 12225


In [15]:
train

id
422668    заехала вчера к бухам (великолепные тетки, в своем деле асы), которые кроме нашей конторы обслуживают еще несколько, в том числе и модельное агентство. -во-от. теперь, когда страничка отсканировалась... нет, кнопку "распознавать" нажимать не надо. когда страничка отсканировалась, нажмите на слов...
456534                                                                                                                                            - я не собираюсь ничего тебе объяснять - почему это, интересно? - ты хочешь, чтобы я объяснил тебе почему я не собираюсь ничего тебе объяснять? что с тобой не так?
402043                                                                        на форуме тв-канала: xxx: да, мне тоже хочется начать в ду же смотреть. просто у них идиотская привычка закупать сериал, что бы потом его не показывать. xxx: и админы, настройте уже цензуру на форуме, я пишу "ду б л я ж", а не " ".
420017                                                             

In [16]:
from transformers import AutoTokenizer
#sberbank-ai/rugpt3large_based_on_gpt2
#sberbank-ai/rugpt3medium_based_on_gpt2
#sberbank-ai/rugpt3small_based_on_gpt2

tokenizer = AutoTokenizer.from_pretrained(model_name)

train_path = 'train_dataset.txt'
test_path = 'test_dataset.txt'

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
from transformers import TextDataset, DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

    test_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset, test_dataset, data_collator

train_dataset, test_dataset, data_collator = load_dataset(train_path, test_path, tokenizer)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:54: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [18]:
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_name)

In [19]:
training_args = TrainingArguments(

    "phrase",
    evaluation_strategy = "epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    
    )


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [21]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,4.114000,3.992858
2,3.982900,3.959330


TrainOutput(global_step=17972, training_loss=4.105968245254337, metrics={'train_runtime': 3493.3741, 'train_samples_per_second': 20.577, 'train_steps_per_second': 5.145, 'total_flos': 4695679107072000.0, 'train_loss': 4.105968245254337, 'epoch': 2.0})

In [22]:
def generate_text(prefix):
    tokens = tokenizer(prefix, return_tensors='pt').to('cuda')
    size = tokens['input_ids'].shape[1]

    output = model.generate(
        **tokens, 
        #end_token=end_token_id,
        do_sample=False,
        max_length=size+50, 
        early_stopping=True,
        length_penalty=2.0,
        repetition_penalty=8., 
        temperature=0.5,
        num_beams=3,
        no_repeat_ngram_size=5
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return prefix + result

In [23]:
print(generate_text("Привет=))"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Привет=))  xxx: а у нас на работе в отделе есть один сотрудник, который постоянно что-то делает. и вот однажды он приходит к нам с просьбой помочь ему сделать ремонт... ну мы его конечно же не стали отговаривать, но все равно


In [24]:
print(generate_text("У меня дома есть старый паяльник"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


У меня дома есть старый паяльник, но он не работает.  xxx: а у нас в школе на первом курсе был препод по информатике и программированию (с) yyy: да ладно тебе! ты же математик-программист? zzz:


In [25]:
print(generate_text("Не спешите выбрасывать старую авторучку"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Не спешите выбрасывать старую авторучку.  xxx: у нас в офисе есть один сотрудник, который не может выйти на работу из-за того, что его уволили за то, что он слишком много времени уделяет работе с бумажками... yyy: ну и


In [26]:
print(generate_text("Вчера пошел погулять"))

Generate config GenerationConfig {
  "bos_token_id": 50256,
  "eos_token_id": 50256,
  "transformers_version": "4.26.1"
}

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Вчера пошел погулять с собакой.  xxx: у нас на работе есть один сотрудник, который работает в нашей компании уже более пяти лет. и вот однажды он приходит ко мне домой после работы (время позднее), а я ему говорю - "ну что ты
